In [3]:
import os
import time
import nltk
import numpy as np
from scipy.sparse import csr_matrix
import operator

In [11]:
# The following code takes all files, replaces new line character with space character and put words of all files in the Words list 
#Reading the directory of our dataset of documents
path = '../data/'

files = os.listdir(path)
print("total Files read: ", len(files))

Words = []
for f in files:
    fhand = open(path+f, "rb")
    file = fhand.read().decode('ISO-8859-1')
    file = file.replace("\n"," ")
    file = file.replace("\r","") #removing line end too
    for word in file.split(" "):
        Words.append(word) #contains all the tokens from all of the files

Words

total Files read:  3000


['Grammatical',
 'Role',
 'Labeling',
 'with',
 'Integer',
 'Linear',
 'Programming',
 'Manfred',
 'Klenner',
 'Institute',
 'of',
 'Computational',
 'Linguistics',
 'University',
 'of',
 'Zurich',
 'klenner@cl.unizh.ch',
 'Abstract',
 'In',
 'this',
 'paper,',
 'we',
 'present',
 'a',
 'formalization',
 'of',
 'grammatical',
 'role',
 'labeling',
 'within',
 'the',
 'framework',
 'of',
 'Integer',
 'Linear',
 'Programming',
 '(ILP).',
 'We',
 'focus',
 'on',
 'the',
 'integration',
 'of',
 'sub-',
 'categorization',
 'information',
 'into',
 'the',
 'deci-',
 'sion',
 'making',
 'process.',
 'We',
 'present',
 'a',
 'first',
 'empirical',
 'evaluation',
 'that',
 'achieves',
 'compet-',
 'itive',
 'precision',
 'and',
 'recall',
 'rates.',
 '1',
 'Introduction',
 'An',
 'often',
 'stressed',
 'point',
 'is',
 'that',
 'the',
 'most',
 'widely',
 'used',
 'classifiers',
 'such',
 'as',
 'Naive',
 'Bayes,',
 'HMM,',
 'and',
 'Memory-based',
 'Learners',
 'are',
 'restricted',
 'to',
 'l

In [12]:
#Note: Removing punctuation and stopword from doc text to avoid unnecessary processing 

def removeStopWords(doc):
    stopset = set(nltk.corpus.stopwords.words('english')) #extract stopwords from nltk corpus.
    tokens = nltk.WordPunctTokenizer().tokenize(doc) #remove punctuations
    clean = [token for token in tokens if token.lower() not in stopset and len(token) > 3  and token.isalpha()] #removed stopwords
    return clean

In [13]:
#find count of unique words in all these files and store it in a dictionary

Word_To_Count = {}
for word in Words:
    if word in Word_To_Count:
        Word_To_Count[word]+=1
    else:
        Word_To_Count[word]=1

print(len(Word_To_Count))


761235


In [15]:
#convert all text into lower case then find count of unique words,
Small_Words = []
#read each file, split all words and append the lower case word in Small_Words list

CleanedDocuments={} #saving tokenized and processed (removed puncuation&stopwords)
    
for f in files:
    fhand = open(path+f, "rb")
    content = fhand.read().decode('ISO-8859-1')
    tokens = removeStopWords(content) #removed stop words and punctuations and return tokens of file text
    smallWordsPerDoc = []
    for word in tokens: 
        if(word.islower()):
            smallWordsPerDoc.append(word) 
    
    Small_Words.extend(smallWordsPerDoc) 
    CleanedDocuments[fhand.name] = smallWordsPerDoc  #saving processed document in CleanedDocuments dictionary to use later
    
print(Small_Words[:100])

['klenner', 'unizh', 'paper', 'present', 'formalization', 'grammatical', 'role', 'labeling', 'within', 'framework', 'focus', 'integration', 'categorization', 'information', 'deci', 'sion', 'making', 'process', 'present', 'first', 'empirical', 'evaluation', 'achieves', 'compet', 'itive', 'precision', 'recall', 'rates', 'often', 'stressed', 'point', 'widely', 'used', 'classifiers', 'based', 'restricted', 'local', 'cisions', 'grammatical', 'role', 'labeling', 'example', 'explicitly', 'express', 'global', 'constraints', 'verb', 'give', 'must', 'arguments', 'particular', 'grammatical', 'role', 'approaches', 'overcome', 'restric', 'tion', 'allow', 'global', 'theory', 'based', 'straints', 'applied', 'apply', 'problem', 'grammatical', 'lation', 'labeling', 'given', 'chunks', 'verb', 'grammatical', 'relation', 'trained', 'maximum', 'entropy', 'classifier', 'vectors', 'phological', 'syntactic', 'positional', 'information', 'output', 'utilized', 'weights', 'ponent', 'generates', 'equations', 'sol

In [18]:
#find count of unique words after converting them to lowercase in all these files and store it in a dictionary
Small_Word_To_Count = {}
for w in Small_Words:
    if w in Small_Word_To_Count:
        Small_Word_To_Count[w]+=1
    else:
        Small_Word_To_Count[w]=1

In [19]:
#sort the small words (in Small_Word_To_Count dictionary) by frequencies and store the results in freq.txt file
#Each line of the file should have word seperated by comma with its counts

sortedSmallWordByFreq = sorted(Small_Word_To_Count.items(),key=lambda item:item[1])

fhand = open("freq.txt",mode='w')
fhand.write('\n'.join('{},{}'.format(x[0],x[1]) for x in sortedSmallWordByFreq))

987728

In [20]:
#sort the words by alphabatical order and store results in words.txt file
#Each line of the file should have word seperated by comma with its counts
fhand = open("words.txt",mode='w')

sortedSmallWordAlphabetically = sorted(Small_Word_To_Count.items(),key=lambda item:item[0])

fhand.write('\n'.join('{},{}'.format(x[0],x[1]) for x in sortedSmallWordAlphabetically))

987728

In [22]:
# Sort the list of Small_Words alphabatically and store it in a new list called Vocabulary
# Vocabulary is the set of unique words
Vocabulary = sorted(list(set(Small_Words))) 
print(len(Vocabulary))
print(Vocabulary[:10])

89904
['aaaa', 'aaab', 'aaabb', 'aaabbb', 'aaaggghhh', 'aaai', 'aaalyser', 'aaap', 'aaarr', 'aabaa']


In [23]:
# Now that the list (Vocabulary) is sorted alphabatically, 
# We can create dictionaries
Word_to_ID = { value: index for index,value in enumerate(Vocabulary) }

print("comparison:::", Word_to_ID["acceptability"], Vocabulary[2])

comparison::: 561 aaabb


In [24]:
# Now inverse the key value pairs of Word_to_ID Dictionary, so that we can search each word by its ID
ID_to_Word =  {value: index for index,value in Word_to_ID.items()}

ID_to_Word

{0: 'aaaa',
 1: 'aaab',
 2: 'aaabb',
 3: 'aaabbb',
 4: 'aaaggghhh',
 5: 'aaai',
 6: 'aaalyser',
 7: 'aaap',
 8: 'aaarr',
 9: 'aabaa',
 10: 'aabb',
 11: 'aabbb',
 12: 'aabbcc',
 13: 'aabbccdd',
 14: 'aabbcd',
 15: 'aabdelali',
 16: 'aach',
 17: 'aachen',
 18: 'aackendoff',
 19: 'aactext',
 20: 'aada',
 21: 'aadcc',
 22: 'aade',
 23: 'aadtr',
 24: 'aaedb',
 25: 'aaee',
 26: 'aage',
 27: 'aagnant',
 28: 'aaiti',
 29: 'aaive',
 30: 'aakkl',
 31: 'aalbessen',
 32: 'aalen',
 33: 'aall',
 34: 'aalto',
 35: 'aame',
 36: 'aanbieding',
 37: 'aand',
 38: 'aaneenges',
 39: 'aanknopingspunt',
 40: 'aanns',
 41: 'aanw',
 42: 'aapl',
 43: 'aard',
 44: 'aardvark',
 45: 'aardvarks',
 46: 'aarinen',
 47: 'aarne',
 48: 'aaron',
 49: 'aarp',
 50: 'aasies',
 51: 'aasociation',
 52: 'aasoeiated',
 53: 'aastriche',
 54: 'aastriiche',
 55: 'aasudee',
 56: 'aata',
 57: 'aatel',
 58: 'aaterlal',
 59: 'aatic',
 60: 'aatlcal',
 61: 'aava',
 62: 'aavashyakataaen',
 63: 'aavgth',
 64: 'aazu',
 65: 'aazut',
 66: 'ab

In [26]:
#### Read each document and save it in a Term Document matrix
#### The Size of document vector should be equal to the size of vocabulary


In [28]:
docToIdDict = { value: index for index, value in enumerate(CleanedDocuments.keys()) } 
idToDoc =  {value: index for index, value in docToIdDict.items()}


termDocFreqMatrix = np.zeros(((len(CleanedDocuments), len(Vocabulary))))

for docId, tokens in CleanedDocuments.items():
    for token in np.unique(tokens):
        try:
            vocabIndex = Word_to_ID[token] 
            docIndex = docToIdDict[docId]
            termDocFreqMatrix[docIndex][vocabIndex] = tokens.count(token)
        except:
            pass

In [29]:
termDocFreqMatrix.shape

(3000, 89904)

In [30]:
#Take Query from the User
Query = input("Enter the search Query: ")
queryTokens = removeStopWords(Query.lower())
queryVector = np.zeros((len(Vocabulary)))

for token in queryTokens:
    try:
        vocabIndex = Word_to_ID[token]
        queryVector[vocabIndex] =  queryTokens.count(token) 
    except:
        pass
#Convert the query into the query vector, This will be same as we converted each document into document vector
#Query vector should have frequency of words
#Take the dot product of query vector with each document vector and rank them, Avoid for loops
#Print the file names
dotProductResult = termDocFreqMatrix.dot(queryVector) 

documentsScore = {idToDoc[i]: value for i, value in enumerate(dotProductResult)}

sortedByScore = sorted(documentsScore.items(), key=operator.itemgetter(1), reverse=True)

sortedByScore

Enter the search Query: hello world


[('../data/E85-1030.pdf.txt', 38.0),
 ('../data/D15-1001.pdf.txt', 34.0),
 ('../data/J00-2002.pdf.txt', 28.0),
 ('../data/D15-1003.pdf.txt', 23.0),
 ('../data/D14-1031.pdf.txt', 23.0),
 ('../data/A83-1013.pdf.txt', 19.0),
 ('../data/E09-2009.pdf.txt', 19.0),
 ('../data/D09-1100.pdf.txt', 17.0),
 ('../data/D14-1043.pdf.txt', 17.0),
 ('../data/D11-1074.pdf.txt', 16.0),
 ('../data/D13-1178.pdf.txt', 16.0),
 ('../data/D15-1204.pdf.txt', 14.0),
 ('../data/D12-1071.pdf.txt', 14.0),
 ('../data/D14-1058.pdf.txt', 13.0),
 ('../data/D15-1224.pdf.txt', 12.0),
 ('../data/D14-1086.pdf.txt', 12.0),
 ('../data/D09-1152.pdf.txt', 12.0),
 ('../data/E09-1092.pdf.txt', 11.0),
 ('../data/E09-2001.pdf.txt', 11.0),
 ('../data/D15-1007.pdf.txt', 11.0),
 ('../data/D13-1038.pdf.txt', 11.0),
 ('../data/E85-1002.pdf.txt', 10.0),
 ('../data/E89-1030.pdf.txt', 10.0),
 ('../data/D12-1034.pdf.txt', 9.0),
 ('../data/D12-1040.pdf.txt', 9.0),
 ('../data/J06-1005.pdf.txt', 9.0),
 ('../data/A00-2042.pdf.txt', 9.0),
 ('..

In [32]:
#Creating New Frequency matrix using sparse matrix approach 

indptr = [0]
indices = []
data = []
vocabulary = {}

for docId, doc in CleanedDocuments.items():
    for term in doc:
        if(term.islower()):
            index = vocabulary.setdefault(term, len(vocabulary))
            indices.append(index)
            data.append(1)
    indptr.append(len(indices))
    
TermFreqSparseMatrix = csr_matrix((data, indices, indptr), dtype=int).toarray()

In [ ]:
# or i can create matrix using existing nd array i created for prev section

TermFreqSparseMatrix = csr_matrix(termDocFreqMatrix, dtype=int).toarray()

In [ ]:
#Take Query from the User
Query = input("Enter the search Query: ")

query2Tokens = removeStopWords(Query.lower())

query2Vector = np.zeros(len(Vocabulary))

for token in query2Tokens:
    try:
        vocabIndex = Word_to_ID[token]
        query2Vector[vocabIndex] =  query2Tokens.count(token) 
    except:
        pass

#Convert the query into the query vector, This will be same as we converted each document into document vector
#Query vector should have frequency of words
#Take the dot product of query vector with each document vector and rank them, Avoid for loops
#Print the file names

sparseDotProductResult = TermFreqSparseMatrix.dot(query2Vector) 

docScoresDict = {idToDoc[i]: value for i, value in enumerate(sparseDotProductResult)}

docDictSortedByScore = sorted(docScoresDict.items(), key=operator.itemgetter(1), reverse=True)

docDictSortedByScore